### Weather model
--------------------------

#### Load the dataset:

In [1]:
# Import libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.api as sm
from scipy import stats
from sklearn import linear_model
from sqlalchemy import create_engine

# Display references
warnings.filterwarnings('ignore')
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Edit pandas display option to show more rows and columns:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# Establish a connection to the dataset and query the dataset:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df = pd.read_sql_query('SELECT * FROM weatherinszeged', con=engine)

# Dispose the connection, as we're only doing a single query:
engine.dispose()

# Print out the first 5 rows of the dataset:
df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472,7.389,0.890,14.120,251.000,15.826,0.000,1015.130,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.356,7.228,0.860,14.265,259.000,15.826,0.000,1015.630,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.378,9.378,0.890,3.928,204.000,14.957,0.000,1015.940,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.289,5.944,0.830,14.104,269.000,15.826,0.000,1016.410,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.756,6.978,0.830,11.045,259.000,15.826,0.000,1016.510,Partly cloudy throughout the day.


#### Build a linear regression model where target variable is the difference between the apparenttemperature and the temperature, and exploratory variables are humidity and windspeed using OLS.

In [3]:
# Create target variable and explanatory variables:
Y = df['target'] = df['temperature'] - df['apparenttemperature']
X = df[['humidity', 'windspeed']]

# Add constant term to the model:
X = sm.add_constant(X)

# Fit the dataset using OLS regression:
result = sm.OLS(Y, X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Thu, 09 Jan 2020   Prob (F-statistic):               0.00
Time:                        18:45:07   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.4381      0.021   -115.948      0.000      -2.479      -2.397
humidity       3.0292      0.024    126.479      0.000       2.982       3.076
windspeed      0.1193      0.001    176.164      0.000       0.118       0.121
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                           0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

* The F-score represents the ratio between a model's unexplained variance compared to a non-feature model. In this weather prediction model, the F-score is 1949 with p value =0, indicating that our features add some information to the reduced model and our model is useful in explaining charges.
* Even though the added features explain the variances in the dataset, the R-squared and adjusted R-squared are very small, which implies that not much of the variance in the outcome variable is explained by features that we pick in the model. In fact, both R-squared and adjusted R-squared in our model are quite low.

#### Include the interaction of humidity and windspeed to the model above and estimate the model using OLS.

In [4]:
# Create the interaction:
df['hum_wind_interaction'] = df['humidity'] * df['windspeed']

Y = df['target']
X = df[['humidity', 'windspeed', 'hum_wind_interaction']]

# Add constant term to the model:
X = sm.add_constant(X)

# Fit the variables to the OLS regression model:
result = sm.OLS(Y, X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Thu, 09 Jan 2020   Prob (F-statistic):               0.00
Time:                        18:45:07   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0839      0.033     -2.511      0.012      -0.149      -0.018
humidity                -0.1775      0.043     -4.133      0.000      -0.262      -0.093
windspeed               -0.0905      0.002    -36.797      0.000      -0.095      -0.086
hum_wind_interaction     0.2971      0.003     88.470      0.000       0.291       0.304
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                           0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

As we add the humidity and windspeed interaction feature in our model, the training performance increases. BIC score decreases, indicating that the SSE(sum of squared errors) decreases. Even though both R-squared and adjusted R-squared statistics increase, which indicates that the second model perform better than the first one we created, the scores are still quite low compared to our desired outcome.

#### Add visibility as an additional explanatory variable to the first model and estimate it.

In [5]:
# Add visibility to our model:
Y = df['target'] 
X = df[['humidity', 'windspeed', 'visibility']]

# Add constant term to the model:
X = sm.add_constant(X)

# Fit the dataset using OLS regression:
result = sm.OLS(Y, X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Thu, 09 Jan 2020   Prob (F-statistic):               0.00
Time:                        18:45:07   Log-Likelihood:            -1.6938e+05
No. Observations:               96453   AIC:                         3.388e+05
Df Residuals:                   96449   BIC:                         3.388e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.5756      0.028    -56.605      0.000      -1.630      -1.521
humidity       2.6066      0.025    102.784      0.000       2.557       2.656
windspeed      0.1199      0.001    179.014      0.000       0.119       0.121
visibility    -0.0540      0.001    -46.614      0.000      -0.056      -0.052
==============================================================================
Omnibus:                     3833.895   Durbin-Watson:                   0.282
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4584.022
Skew:                           0.459   Prob(JB):                         0.00
Kurtosis:                       3.545   Cond. No.                         131.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

* R-squared and adjusted R-squared of the model is about 3.0, both increase compared to the first model. Adding visibility to the model improve the training performance. However, this improvement is lower than the second model that is achieved by putting the interaction term in the model. Hence, we conclude that interaction term contributes more to the model. From this point of view, interaction term can be regarded as more important than the visibility in explaining the variance in the target.
* As we know, the lower values the better for information criterions like AIC and BIC. The first model's AIC and BIC values are 340,900. The second model's are 333,400 and the third model's are 338,800. Among these, the model with the lowest AIC and BIC scores is the second model. So, the best of these three models is the second one.

### House price model
---------------------------------

#### Load the dataset:

In [6]:
# Query the database to extract dataset:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

houseprices_df = pd.read_sql_query('SELECT * FROM houseprices', con=engine)

# Dispose the connection, as we're only doing a single query:
engine.dispose()

# Print out the head of the dataset:
houseprices_df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,heatingqc,centralair,electrical,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.000,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.000,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.000,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.000,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.000,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.000,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.000,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.000,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.000,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.000,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal,250000


In [7]:
# Convert categorical variables to dummy variables:
houseprices_df['centralair'] = pd.get_dummies(houseprices_df['centralair'], drop_first=True)
houseprices_df['centralair'].head()

0    1
1    1
2    1
3    1
4    1
Name: centralair, dtype: uint8

In [8]:
# Define the target variable and the explanatory variables:
Y1 = houseprices_df['saleprice']
X1 = houseprices_df[['overallqual', 'grlivarea', 'garagecars', 'centralair']]

# Manually add a constant in statsmodels' sm
X1 = sm.add_constant(X1)

# Fit the variables to the regression model
result = sm.OLS(Y1, X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.741
Method:                 Least Squares   F-statistic:                     1042.
Date:                Thu, 09 Jan 2020   Prob (F-statistic):               0.00
Time:                        18:45:09   Log-Likelihood:                -17557.
No. Observations:                1460   AIC:                         3.512e+04
Df Residuals:                    1455   BIC:                         3.515e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.092e+05   5658.917    -19.305      0.000    -1.2e+05   -9.81e+04
overallqual  2.635e+04   1089.520     24.182      0.000    2.42e+04    2.85e+04
grlivarea      51.6199      2.556     20.196      0.000      46.606      56.634
garagecars   2.061e+04   1810.725     11.381      0.000    1.71e+04    2.42e+04
centralair   1.586e+04   4505.540      3.521      0.000    7024.428    2.47e+04
==============================================================================
Omnibus:                      439.032   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10183.651
Skew:                           0.846   Prob(JB):                         0.00
Kurtosis:                      15.827   Cond. No.                     9.69e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.69e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

* F-score of the model is quite high, and since p-value =0, we can be confident that our features add some information to the reduced model and our model is useful in explaining charges.
* Both R-squared and adjusted R-squared are around 74%, indicating that our model is quite good at prediciting the saleprice. We don't want R-squared and adjusted R-squared to be too high, as it could mean our model suffers from overfitting. However, 74% is still lower than our desired outcome in this case.

In [12]:
# Convert 'mzoning' to dummy variables:
houseprices_df = pd.concat([houseprices_df,pd.get_dummies(houseprices_df['mszoning'], prefix='mszoning', drop_first=True)], axis=1)
houseprices_df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,heatingqc,centralair,electrical,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice,totalsf,int_over_sf,mszoning_FV,mszoning_RH,mszoning_RL,mszoning_RM
0,1,60,RL,65.000,8450,1,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.000,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,1,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.000,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal,208500,2566,17962,0,0,1,0
1,2,20,RL,80.000,9600,1,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.000,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,1,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.000,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal,181500,2524,15144,0,0,1,0
2,3,60,RL,68.000,11250,1,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.000,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,1,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.000,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal,223500,2706,18942,0,0,1,0
3,4,70,RL,60.000,9550,1,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.000,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,1,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.000,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml,140000,2473,17311,0,0,1,0
4,5,60,RL,84.000,14260,1,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.000,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,1,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal,250000,3343,26744,0,0,1,0


In [19]:
# Include the feature interactions to our model:
houseprices_df['totalsf'] = houseprices_df['totalbsmtsf'] + houseprices_df['firstflrsf'] + houseprices_df['secondflrsf']
houseprices_df['int_over_sf'] = houseprices_df['totalsf'] * houseprices_df['overallqual']

# Define the target variable and the explanatory variables:
Y1 = houseprices_df['saleprice']
X1 = houseprices_df[['overallqual', 'grlivarea', 'garagecars', 'int_over_sf', 'overallcond', 
                     'mszoning_FV', 'mszoning_RL', ]]

# Manually add a constant in statsmodels' sm
X1 = sm.add_constant(X1)

# Fit the variables to the regression model
result = sm.OLS(Y1, X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.785
Model:                            OLS   Adj. R-squared:                  0.784
Method:                 Least Squares   F-statistic:                     755.8
Date:                Thu, 09 Jan 2020   Prob (F-statistic):               0.00
Time:                        18:51:36   Log-Likelihood:                -17423.
No. Observations:                1460   AIC:                         3.486e+04
Df Residuals:                    1452   BIC:                         3.490e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -5.807e+04   8575.405     -6.772      0.000   -7.49e+04   -4.12e+04
overallqual   1.08e+04   1462.847      7.380      0.000    7925.797    1.37e+04
grlivarea      13.7302      3.419      4.016      0.000       7.024      20.437
garagecars    1.86e+04   1680.964     11.065      0.000    1.53e+04    2.19e+04
int_over_sf     4.6252      0.317     14.598      0.000       4.004       5.247
overallcond  4581.3577    900.891      5.085      0.000    2814.171    6348.544
mszoning_FV  2.383e+04   5425.504      4.393      0.000    1.32e+04    3.45e+04
mszoning_RL  2.149e+04   2720.159      7.899      0.000    1.62e+04    2.68e+04
==============================================================================
Omnibus:                     1087.180   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           211640.924
Skew:                          -2.499   Prob(JB):                         0.00
Kurtosis:                      61.771   Cond. No.                     1.67e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

It looks like our second model performs better than the first model. Both R-squared and adjusted R-squared increase from the first model, and both AIC and BIC score decrease, indicating a lower SSE.